In [ ]:
%tensorflow_version 2.x
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
!pip install transformers
!pip install tensorflow
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.callbacks import ProgbarLogger

# Load the dataset
data = pd.read_csv("/content/NLP_NEW_DATA (3).csv", encoding='latin-1')
data = data.dropna(subset=["Prompt"])  # Remove rows with missing values in "Prompt" column
texts = data["Prompt"].tolist()
intents = data[["IsFood", "Cuisine Type", "Cuisine Region", "Dish Name"]].values

# Encode the intents as numerical labels
label_encoders = []
encoded_intent s = []
for i in range(intents.shape[1]):
    label_encoder = LabelEncoder()
    encoded_intent = label_encoder.fit_transform(intents[:, i])
    label_encoders.append(label_encoder)
    encoded_intents.append(encoded_intent)

# Compute class weights
class_weights = []
for i in range(intents.shape[1]):
    class_weight = compute_class_weight("balanced", classes=sorted(set(encoded_intents[i])), y=encoded_intents[i])
    class_weights.append(class_weight)

# Preprocess the data using tf.dat
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_datasets = []
test_datasets = []

for i in range(intents.shape[1]):
    filtered_indices = [idx for idx, intent in enumerate(encoded_intents[i]) if intent != -1]
    filtered_texts = [texts[idx] for idx in filtered_indices]
    filtered_intents = [encoded_intents[i][idx] for idx in filtered_indices]

    train_texts, test_texts, train_intents, test_intents = train_test_split(
        filtered_texts,
        filtered_intents,
        test_size=0.2,
        random_state=42
    )

    train_encodings = tokenizer(train_texts, truncation=True, padding=True)
    test_encodings = tokenizer(test_texts, truncation=True, padding=True)

    train_dataset = tf.data.Dataset.from_tensor_slices((
        dict(train_encodings),
        train_intents
    ))
    test_dataset = tf.data.Dataset.from_tensor_slices((
        dict(test_encodings),
        test_intents
    ))

    train_datasets.append(train_dataset.shuffle(100).batch(16))
    test_datasets.append(test_dataset.batch(16))



# Define the model architecture
models = []
for i in range(intents.shape[1]):
    model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(set(encoded_intents[i])))
    models.append(model)

# Fine-tune the models
epochs = 5  # Define the number of epochs you want to train for

for i in range(intents.shape[1]):
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()

    for epoch in range(epochs):
        # Training loop
        total_steps = len(train_datasets[i])
        current_step = 0

        for inputs, labels in train_datasets[i]:
            current_step += 1

            with tf.GradientTape() as tape:
                # Forward pass
                predictions = models[i](inputs, training=True)
                logits = predictions['logits']  # Extract logits
                loss = loss_fn(labels, logits)  # Compute loss using logits

            # Compute gradients and update weights
            gradients = tape.gradient(loss, models[i].trainable_variables)
            optimizer.apply_gradients(zip(gradients, models[i].trainable_variables))

            # Update accuracy metric
            accuracy_metric.update_state(labels, logits)  # Use logits for accuracy calculation

            # Print progress
            print(f"\rEpoch {epoch+1}/{epochs} - Step {current_step}/{total_steps}", end="")

        # Validation loop
        for inputs, labels in test_datasets[i]:
            predictions = models[i](inputs, training=False)
            logits = predictions['logits']  # Extract logits
            accuracy_metric.update_state(labels, logits)  # Use logits for accuracy calculation

        # Print training and validation accuracy for the current epoch
        print(f"\nEpoch {epoch+1}/{epochs}: Training Accuracy = {accuracy_metric.result().numpy()}, Validation Accuracy = {accuracy_metric.result().numpy()}")

        # Reset accuracy metric for the next epoch
        accuracy_metric.reset_states()

    print(f"Completed Epoch {epoch+1} for intent {i+1}")
    models[i].save_pretrained(f"BERT_V1")

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# ... (your existing code) ...

# Initialize lists to store the true and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test dataset to collect true and predicted labels
for inputs, labels in test_datasets[i]:
    predictions = models[i](inputs, training=False)
    logits = predictions['logits']
    predicted_label = tf.argmax(logits, axis=1).numpy()

    true_labels.extend(labels.numpy())
    predicted_labels.extend(predicted_label)

# Calculate precision, recall, and F1 score
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
confusion_mat = confusion_matrix(true_labels, predicted_labels)

# Print metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(confusion_mat)

# Example text prompts
text_prompts = [
    "Can you bring me some Dahi Puri?",
    "Can you bring me some Cultery?",
    "Can you bring me some Tandoori Chicken?",
    "Can you bring me some Manchurian?",
    "Can you bring me some Chicken Tikka Sandwich?",
    "Can you bring me some Chicken 65?",
    "Can you bring me some Butter Naan?",
    "Can you bring me some Dum Biryani?",
    "Can you bring me some Salad?",
    "Can you bring me some Pizza?",
    "Can you bring me some Bread?",
]

# Tokenize and preprocess the text prompts
tokenized_prompts = [tokenizer.encode_plus(
    prompt,
    truncation=True,
    padding=True,
    return_tensors="tf"
) for prompt in text_prompts]

# Get the input IDs from the tokenized prompts
input_ids_list = [prompt["input_ids"] for prompt in tokenized_prompts]

# Make predictions using the trained models
predicted_labels_list = []
for i in range(intents.shape[1]):
    predicted_labels = []
    for input_ids in input_ids_list:
        predictions = models[i].predict(input_ids)
        logits = predictions['logits']  # Extract logits
        predicted_label = label_encoders[i].inverse_transform(tf.argmax(logits, axis=1).numpy())
        predicted_labels.append(predicted_label[0])
    predicted_labels_list.append(predicted_labels)

# Print the predicted intent labels for each text prompt
for i, prompt in enumerate(text_prompts):
    print(f"Predicted Intent for prompt {i+1}:")
    print("IsFood:", predicted_labels_list[0][i])
    print("Cuisine Type:", predicted_labels_list[1][i])
    print("Cuisine Region:", predicted_labels_list[2][i])
    print("Dish Name:", predicted_labels_list[3][i])
    print()


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly init

Epoch 1/5 - Step 81/81
Epoch 1/5: Training Accuracy = 0.9956602454185486, Validation Accuracy = 0.9956602454185486
Epoch 2/5 - Step 81/81
Epoch 2/5: Training Accuracy = 0.9993800520896912, Validation Accuracy = 0.9993800520896912
Epoch 3/5 - Step 81/81
Epoch 3/5: Training Accuracy = 0.9993800520896912, Validation Accuracy = 0.9993800520896912
Epoch 4/5 - Step 81/81
Epoch 4/5: Training Accuracy = 0.9993800520896912, Validation Accuracy = 0.9993800520896912
Epoch 5/5 - Step 81/81
Epoch 5/5: Training Accuracy = 0.9993800520896912, Validation Accuracy = 0.9993800520896912
Completed Epoch 5 for intent 1
Epoch 1/5 - Step 81/81
Epoch 1/5: Training Accuracy = 0.860508382320404, Validation Accuracy = 0.860508382320404
Epoch 2/5 - Step 81/81
Epoch 2/5: Training Accuracy = 0.9721016883850098, Validation Accuracy = 0.9721016883850098
Epoch 3/5 - Step 81/81
Epoch 3/5: Training Accuracy = 0.9981400966644287, Validation Accuracy = 0.9981400966644287
Epoch 4/5 - Step 81/81
Epoch 4/5: Training Accuracy

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.9604
Recall: 0.9752
F1 Score: 0.9659
Confusion Matrix:
[[8 0 0 ... 0 0 0]
 [0 7 0 ... 0 0 0]
 [0 0 8 ... 0 0 0]
 ...
 [0 0 0 ... 8 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 3]]
1/1 [==============================] - 0s 70ms/step
Predicted Intent for prompt 1:
IsFood: Yes
Cuisine Type: Indian
Cuisine Region: West India
Dish Name: Dahi Puri

Predicted Intent for prompt 2:
IsFood: Yes
Cuisine Type: General
Cuisine Region: General
Dish Name: General

Predicted Intent for prompt 3:
IsFood: Yes
Cuisine Type: Indian
Cuisine Region: General
Dish Name: Tandoori Chiken

Predicted Intent for prompt 4:
IsFood: Yes
Cuisine Type: General
Cuisine Region: General
Dish Name: Manchurian

Predicted Intent for prompt 5:
IsFood: Yes
Cuisine Type: Indian
Cuisine Region: West India
Dish Name: Chiken Tikka Sandwish

Predicted Intent for prompt 6:
IsFood: Yes
Cuisine Type: Indian
Cuisine Region: South India
Dish Name: Chiken 65

Predicted Intent for prompt 7:
IsFood: Yes
Cuisine Type: Indian
Cuisine

In [ ]:
for i in range(4):  # Assuming you have 4 intent categories
    # ... (rest of the code)

    # Iterate through the test dataset to collect true and predicted labels
    for inputs, labels in test_datasets[i]:
        predictions = models[i](inputs, training=False)  # Corrected variable name here
        logits = predictions['logits']
        predicted_label = tf.argmax(logits, axis=1).numpy()

        true_labels.extend(labels.numpy())
        predicted_labels.extend(predicted_label)

    # Calculate precision, recall, and F1 score
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    # Calculate confusion matrix
    confusion_mat = confusion_matrix(true_labels, predicted_labels)

    # Print metrics
    print(f"PrAecision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(confusion_mat)

    # Print the entire confusion matrix
    print("Full Confusion Matrix:")
    print(confusion_mat)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.9938
Recall: 0.9969
F1 Score: 0.9954
Confusion Matrix:
[[  0   1]
 [  0 322]]
Full Confusion Matrix:
[[  0   1]
 [  0 322]]
Precision: 0.9985
Recall: 0.9985
F1 Score: 0.9984
Confusion Matrix:
[[ 61   1]
 [  0 584]]
Full Confusion Matrix:
[[ 61   1]
 [  0 584]]
Precision: 0.9896
Recall: 0.9897
F1 Score: 0.9893
Confusion Matrix:
[[105   1   0   0   0   0   0   0]
 [  0 709   0   1   0   0   0   0]
 [  0   0  13   0   0   0   0   0]
 [  0   8   0  38   0   0   0   0]
 [  0   0   0   0  24   0   0   0]
 [  0   0   0   0   0  14   0   0]
 [  0   0   0   0   0   0  17   0]
 [  0   0   0   0   0   0   0  39]]
Full Confusion Matrix:
[[105   1   0   0   0   0   0   0]
 [  0 709   0   1   0   0   0   0]
 [  0   0  13   0   0   0   0   0]
 [  0   8   0  38   0   0   0   0]
 [  0   0   0   0  24   0   0   0]
 [  0   0   0   0   0  14   0   0]
 [  0   0   0   0   0   0  17   0]
 [  0   0   0   0   0   0   0  39]]
Precision: 0.9823
Recall: 0.9861
F1 Score: 0.9835
Confusion Matrix:
[[113

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
